In [4]:
import polars as pl

In [10]:
df = pl.read_parquet("../data/ledger.parquet")

In [11]:
(
    df
    .select(
        pl.exclude('mcc'),
        mcc=pl.col('mcc').shuffle(),
    )
    .write_parquet("../data/shuffled-mcc-ledger.parquet")
)